In [25]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.offline import iplot, init_notebook_mode
import plotly.io as pio
import glob, os, sys
import logging
from sys import argv
import re
from scipy.ndimage.filters import gaussian_filter1d
from scipy.interpolate import Rbf, InterpolatedUnivariateSpline
from scipy.signal import savgol_filter

# path = '/Users/weugene/basilisk/work/tube/res18/'
# case = 'a'
path = '/Users/weugene/basilisk/work/tube/tmp/poisson/res18/'
case = 'a'
# path = '/Users/weugene/basilisk/work/tube/tmp/poisson/res19/'
# case = 'b'
# path = '/Users/weugene/basilisk/work/tube/res21/dumps/'
# case = 'd'
# path = '/Users/weugene/basilisk/work/tube/tmp/poisson/res19/'
# case = 'b'
# path = '/Users/weugene/basilisk/work/tube/tmp/euler/res20/'
# case = 'c'
# path = '/Users/weugene/basilisk/work/tube/res20/'
# case = 'c'
# path = '/Users/weugene/basilisk/work/tube/tmp/landau/res21/'
# case = 'd'
# path = '/Users/weugene/basilisk/work/tube/res22_corse2refined/'
# case = 'e'
# path = '/Users/weugene/basilisk/work/tube/tmp/poisson/res22_adaptffsu/'
# case = 'e'

path_ex = '/Users/weugene/basilisk/work/tube/'

x_name = 'Points_0'
y_name = 'Points_1'
# file_name = ["data_i=","_t=",".csv"]
# file_name = ["tracer","_t=",".csv"]
file_name = ["contour","_t=",".csv"]
list_of_consideration = range(0,137,1)
# list_of_consideration = [1, 30, 60, 96]
diam = 0.514
# x_name = 'Points:0'
# y_name = 'Points:1'
### STYLE
yaxis = dict(
              tickfont = dict(
              family = 'Times New Roman',
              size = 20,
              color = 'black'
              ),
              titlefont = dict(
                  family = 'Times New Roman',
                  size = 25,
                  color = 'black'
              ),
          )
xaxis = dict(
              tickfont = dict(
                  family = 'Times New Roman',
                  size = 20,
                  color = 'black'
              ),
              titlefont = dict(
                  family = 'Times New Roman',
                  size = 25,
                  color = 'black'
              )
          )
axis_style = dict(showline=True, gridwidth=1, gridcolor='lightgrey', linewidth=2, linecolor='black', mirror=True, ticks='outside',
                 tickfont = dict(family = 'Times New Roman', size = 18, color = 'black'))
bg_style = {'plot_bgcolor': 'rgba(0, 0, 0, 0)', 'paper_bgcolor': 'rgba(0, 0, 0, 0)',}
### END of STYLE
print ("path:", path, "\npath_ex:", path_ex)
files_csv = []
files_csv += glob.glob(path + file_name[0]+"*"+file_name[1]+"*"+file_name[2])
print("FOUND:",files_csv)
assert len(files_csv) > 0, "Empty files_csv"

files_csv=[ os.path.basename(x) for x in files_csv]

files_nums=[]
for i,x in enumerate(files_csv):
    match_number = re.compile('-?\ *[0-9]+\.?[0-9]*(?:[Ee]\ *-?\ *[0-9]+)?')
    final_list = re.findall(match_number, x)
    files_nums.append((int(i), float(final_list[0]), x))
print("files_nums:",files_nums)
files_nums=sorted(files_nums, key=lambda x: x[1])
print ("FILES_NUMS after soting:",files_nums)
files_csv_sorted_filtered = []
files_csv = []
for (i,x) in enumerate(files_nums):
    if x[0] in list_of_consideration:
        files_csv_sorted_filtered.append((x[0], x[1] , path + x[2]))
        files_csv.append(path + x[2])
        print("Files csv are reading...", x[2])

Nf = len(files_csv)
def name_generator(s):
#     return  [path_ex + '10' + s + '.csv', path_ex + '10' + s + '_Fluent.csv', path_ex + '10' + s + '_OpenFoam.csv' ], ['10' + s + ' experiment', '10' + s + ' Fluent','10' + s + ' OpenFoam', '10' + s + ' BP']
    return  [path_ex + '10' + s + '.csv', path_ex + '10' + s + '_Fluent_neighbors.csv', path_ex + '10' + s + '_OpenFoam.csv' ], ['10' + s + ' experiment', '10' + s + ' Fluent','10' + s + ' OpenFoam', '10' + s + ' BP']
files_csv_exp, names = name_generator(case)

col =   ['blue',           'red',        'hsv(120,100,100)',       'black' ]
# files_csv_exp = ['a.csv','b.csv','c.csv']
print('files_csv:',files_csv)
print('files_csv_exp:',files_csv_exp)
df = []
df_exp = []
# params = ['10a.csv':'']


for i, file in enumerate(files_csv_exp):
    df_exp.append(pd.read_csv(file, header=None))
    df_exp[i].columns = [x_name, y_name] 
    df_exp[i][x_name] /= diam
    df_exp[i][y_name] /= diam
i_exp = 0
for i, file in enumerate(files_csv):
#     if i%5 != 0:
#         print("FILE MISSED:", file)
#         continue
        
    df.append(pd.read_csv(file))  
    mdf = df[i]
    j = str(files_csv_sorted_filtered[i][0])
    t = str(files_csv_sorted_filtered[i][1])
    print('i=', j, ' t=', t,' file=', file)
#     display(mdf.head())
#     N = mdf.shape[0]
#     dfdown = mdf[(mdf[y_name]<0)].sort_values(by=[x_name, y_name])
#     dfup = mdf[(mdf[y_name]>0)].sort_values(by=[x_name, y_name], ascending=False)
#     mdf = pd.concat([dfdown, dfup, dfdown.head(1)])
    x = mdf[x_name][mdf[y_name]>=0]
    y = mdf[y_name][mdf[y_name]>=0]
    aa = zip(x,y)
    aa = sorted(aa, key=lambda item: (item[0]))
    aa = zip(*aa)
    x = np.array(x)
    y = np.array(y)

    minx = min(x)
    maxx = max(x)
    rmax = max(y)
    try:
        x_rmax = x[y == rmax][0] - minx
    except:
        x_rmax = x[y == rmax] - minx
    
    x_cm = (x-minx).mean() # center of mass
    print("minx=",minx, " maxx=", maxx, "x_cm", x_cm)
    print("Min Thickness=", 0.5 - max(y))
    print("Mean Thickness=", 0.5 - y[np.logical_and(x > x_rmax + minx, x < x_cm + minx)].mean())
    
    
    x_exp = np.array(df_exp[i_exp][x_name])
    y_exp = np.array(df_exp[i_exp][y_name])
    minx_exp = min(x_exp)
    rmax_exp = max(y_exp)
    try:
        x_rmax_exp = x_exp[y_exp == rmax_exp][0] - minx_exp
    except:
        x_rmax_exp = x_exp[y_exp == rmax_exp] - minx_exp
    
    x_cm_exp = (x_exp-minx_exp).mean() # center of mass
    print("Min Thickness_exp=", 0.5 - max(y_exp))

    print("Mean Thickness_exp=", 0.5 - y_exp[np.logical_and(x_exp > x_rmax_exp + minx_exp, x_exp < x_cm_exp + minx_exp)].mean())


    
    if True:
        fig = go.Figure(go.Scatter(x=x-minx, y=y, name='t=' + t,
                                 mode='markers',
                                 marker = dict(
                                     color = col[-1],
                                     size = 2
        #                                  ,
        #                              line = dict(
        #                               color = 'rgb(231, 99, 250)',
        #                               width = 12
        #                               )
                                 ),
                                 textfont=dict(
                                    family="sans serif",
                                    size=18,
                                    color="LightSeaGreen")
                                 ))
#         fig.add_trace(go.Scatter(x=[x_cm,x_cm], y=[0,1], name='x_cm',
#                          mode='lines',
#                          line=dict(color='black', width=2, dash='dot'),
#                          textfont=dict(
#                             family="sans serif",
#                             size=18,
#                             color="LightSeaGreen")
#                          ), 
#                         )
    
#         fig.add_trace(go.Scatter(x=[x_rmax,x_rmax], y=[0,1], name='x_rmax=',
#                          mode='lines',
#                          line=dict(color='black', width=2, dash='dot'),
#                          textfont=dict(
#                             family="sans serif",
#                             size=18,
#                             color="LightSeaGreen")
#                          ))
        
#         fig.add_trace(go.Scatter(
#             x=[0, maxx - minx],
#             y=[0.5, 0.5],
#             mode='lines',
#             line=dict(color='black', width=1),
#             showlegend=False)
#         )
    
    
#         fig.add_trace(go.Scatter(
#             x=[0, maxx - minx],
#             y=[-0.5, -0.5],
#             mode='lines',
#             line=dict(color='black', width=1),
#             showlegend=False)
#         )
        
        for i_exp in range(0,3):
            fig.add_trace(go.Scatter(x=df_exp[i_exp][x_name], y=df_exp[i_exp][y_name], name=names[i_exp],
                                 mode='lines',
                                 marker = dict(
                                     color = col[i_exp],
                                     size = 5
    #                                  ,
        #                              line = dict(
        #                               color = 'rgb(231, 99, 250)',
        #                               width = 12
        #                               )
                                 ),
                                 textfont=dict(
                                    family="sans serif",
                                    size=18,
                                    color="LightSeaGreen")
                                 ))
        fig.update_layout(
            width = 1000,
            height = 500,
            xaxis_title='x',
            yaxis_title='y',
            yaxis = yaxis,
            xaxis = xaxis
        )
        fig.update_layout(bg_style)
        fig.update_xaxes(axis_style)
        fig.update_yaxes(axis_style)
        
#         fig.update_xaxes(range=[0,2.5])
        fig.update_yaxes(range=[0,0.5])
        fig.update_layout(showlegend=False)
        
        
        if True:
            fn = path + 'M_bubble_t=' + t + '.pdf'
            iplot(fig)
            pio.write_image(fig, fn)
            print("Successfully generated:", fn)

path: /Users/weugene/basilisk/work/tube/tmp/poisson/res18/ 
path_ex: /Users/weugene/basilisk/work/tube/
FOUND: ['/Users/weugene/basilisk/work/tube/tmp/poisson/res18/contour_t=10.0353.csv', '/Users/weugene/basilisk/work/tube/tmp/poisson/res18/contour_t=10.0156.csv', '/Users/weugene/basilisk/work/tube/tmp/poisson/res18/contour_t=10.0221.csv', '/Users/weugene/basilisk/work/tube/tmp/poisson/res18/contour_t=10.0093.csv', '/Users/weugene/basilisk/work/tube/tmp/poisson/res18/contour_t=10.1003.csv', '/Users/weugene/basilisk/work/tube/tmp/poisson/res18/contour_t=10.0672.csv', '/Users/weugene/basilisk/work/tube/tmp/poisson/res18/contour_t=10.0288.csv', '/Users/weugene/basilisk/work/tube/tmp/poisson/res18/contour_t=0.159121.csv', '/Users/weugene/basilisk/work/tube/tmp/poisson/res18/contour_t=10.0039.csv', '/Users/weugene/basilisk/work/tube/tmp/poisson/res18/contour_t=9.99884.csv', '/Users/weugene/basilisk/work/tube/tmp/poisson/res18/contour_t=10.1335.csv']
files_nums: [(0, 10.0353, 'contour_t=10.

Successfully generated: /Users/weugene/basilisk/work/tube/tmp/poisson/res18/M_bubble_t=0.159121.pdf
i= 9  t= 9.99884  file= /Users/weugene/basilisk/work/tube/tmp/poisson/res18/contour_t=9.99884.csv
minx= 11.9747  maxx= 19.998 x_cm 1.973308040201005
Min Thickness= 0.0
Mean Thickness= nan
Min Thickness_exp= 0.013175756409622708
Mean Thickness_exp= 0.017307877628702006


/Users/weugene/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:156: RuntimeWarning:

Mean of empty slice.



Successfully generated: /Users/weugene/basilisk/work/tube/tmp/poisson/res18/M_bubble_t=9.99884.pdf
i= 8  t= 10.0039  file= /Users/weugene/basilisk/work/tube/tmp/poisson/res18/contour_t=10.0039.csv
minx= 11.9785  maxx= 19.998 x_cm 1.9645257396449702
Min Thickness= 0.0
Mean Thickness= nan
Min Thickness_exp= 0.013175756409622708
Mean Thickness_exp= 0.017307877628702006


Successfully generated: /Users/weugene/basilisk/work/tube/tmp/poisson/res18/M_bubble_t=10.0039.pdf
i= 3  t= 10.0093  file= /Users/weugene/basilisk/work/tube/tmp/poisson/res18/contour_t=10.0093.csv
minx= 11.982000000000001  maxx= 19.9975 x_cm 1.9201184438040333
Min Thickness= 0.0
Mean Thickness= nan
Min Thickness_exp= 0.013175756409622708
Mean Thickness_exp= 0.017307877628702006


...

Successfully generated: /Users/weugene/basilisk/work/tube/tmp/poisson/res18/M_bubble_t=10.0288.pdf
i= 0  t= 10.0353  file= /Users/weugene/basilisk/work/tube/tmp/poisson/res18/contour_t=10.0353.csv
minx= 12.011  maxx= 19.997 x_cm 1.940249506903354
Min Thickness= 0.0
Mean Thickness= nan
Min Thickness_exp= 0.013175756409622708
Mean Thickness_exp= 0.017307877628702006


Successfully generated: /Users/weugene/basilisk/work/tube/tmp/poisson/res18/M_bubble_t=10.0353.pdf
i= 5  t= 10.0672  file= /Users/weugene/basilisk/work/tube/tmp/poisson/res18/contour_t=10.0672.csv
minx= 12.048  maxx= 19.9965 x_cm 1.9124875835721107
Min Thickness= 0.0
Mean Thickness= nan
Min Thickness_exp= 0.013175756409622708
Mean Thickness_exp= 0.017307877628702006


Successfully generated: /Users/weugene/basilisk/work/tube/tmp/poisson/res18/M_bubble_t=10.0672.pdf
i= 4  t= 10.1003  file= /Users/weugene/basilisk/work/tube/tmp/poisson/res18/contour_t=10.1003.csv
minx= 12.0845  maxx= 19.999000000000002 x_cm 1.9656180244399186
Min Thickness= 0.0
Mean Thickness= nan
Min Thickness_exp= 0.013175756409622708
Mean Thickness_exp= 0.017307877628702006


Successfully generated: /Users/weugene/basilisk/work/tube/tmp/poisson/res18/M_bubble_t=10.1003.pdf
i= 10  t= 10.1335  file= /Users/weugene/basilisk/work/tube/tmp/poisson/res18/contour_t=10.1335.csv
minx= 12.1219  maxx= 19.9984 x_cm 1.971193121149897
Min Thickness= 0.0
Mean Thickness= nan
Min Thickness_exp= 0.013175756409622708
Mean Thickness_exp= 0.017307877628702006


Successfully generated: /Users/weugene/basilisk/work/tube/tmp/poisson/res18/M_bubble_t=10.1335.pdf


In [71]:
import numpy as np

def non_uniform_savgol(x, y, window, polynom):
    """
    Applies a Savitzky-Golay filter to y with non-uniform spacing
    as defined in x

    This is based on https://dsp.stackexchange.com/questions/1676/savitzky-golay-smoothing-filter-for-not-equally-spaced-data
    The borders are interpolated like scipy.signal.savgol_filter would do

    Parameters
    ----------
    x : array_like
        List of floats representing the x values of the data
    y : array_like
        List of floats representing the y values. Must have same length
        as x
    window : int (odd)
        Window length of datapoints. Must be odd and smaller than x
    polynom : int
        The order of polynom used. Must be smaller than the window size

    Returns
    -------
    np.array of float
        The smoothed y values
    """
    if len(x) != len(y):
        raise ValueError('"x" and "y" must be of the same size')

    if len(x) < window:
        raise ValueError('The data size must be larger than the window size')

    if type(window) is not int:
        raise TypeError('"window" must be an integer')

    if window % 2 == 0:
        raise ValueError('The "window" must be an odd integer')

    if type(polynom) is not int:
        raise TypeError('"polynom" must be an integer')

    if polynom >= window:
        raise ValueError('"polynom" must be less than "window"')

    half_window = window // 2
    polynom += 1

    # Initialize variables
    A = np.empty((window, polynom))     # Matrix
    tA = np.empty((polynom, window))    # Transposed matrix
    t = np.empty(window)                # Local x variables
    y_smoothed = np.full(len(y), np.nan)

    # Start smoothing
    for i in range(half_window, len(x) - half_window, 1):
        # Center a window of x values on x[i]
        for j in range(0, window, 1):
            t[j] = x[i + j - half_window] - x[i]

        # Create the initial matrix A and its transposed form tA
        for j in range(0, window, 1):
            r = 1.0
            for k in range(0, polynom, 1):
                A[j, k] = r
                tA[k, j] = r
                r *= t[j]

        # Multiply the two matrices
        tAA = np.matmul(tA, A)

        # Invert the product of the matrices
        tAA = np.linalg.inv(tAA)

        # Calculate the pseudoinverse of the design matrix
        coeffs = np.matmul(tAA, tA)

        # Calculate c0 which is also the y value for y[i]
        y_smoothed[i] = 0
        for j in range(0, window, 1):
            y_smoothed[i] += coeffs[0, j] * y[i + j - half_window]

        # If at the end or beginning, store all coefficients for the polynom
        if i == half_window:
            first_coeffs = np.zeros(polynom)
            for j in range(0, window, 1):
                for k in range(polynom):
                    first_coeffs[k] += coeffs[k, j] * y[j]
        elif i == len(x) - half_window - 1:
            last_coeffs = np.zeros(polynom)
            for j in range(0, window, 1):
                for k in range(polynom):
                    last_coeffs[k] += coeffs[k, j] * y[len(y) - window + j]

    # Interpolate the result at the left border
    for i in range(0, half_window, 1):
        y_smoothed[i] = 0
        x_i = 1
        for j in range(0, polynom, 1):
            y_smoothed[i] += first_coeffs[j] * x_i
            x_i *= x[i] - x[half_window]

    # Interpolate the result at the right border
    for i in range(len(x) - half_window, len(x), 1):
        y_smoothed[i] = 0
        x_i = 1
        for j in range(0, polynom, 1):
            y_smoothed[i] += last_coeffs[j] * x_i
            x_i *= x[i] - x[-half_window - 1]

    return y_smoothed

In [73]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.offline import iplot, init_notebook_mode
import plotly.io as pio
import glob, os, sys
import logging
from sys import argv
import re
from scipy.ndimage.filters import gaussian_filter1d
from scipy.interpolate import Rbf, InterpolatedUnivariateSpline
from scipy.interpolate import splev, splrep
from scipy.signal import savgol_filter
from statistics import mean

def nearest_point(x, y, x0, y0):
    values = (x - x0)**2 + (y - y0)**2
    index_min = np.argmin(values)
    if values[index_min] >0.1:
        print("ERROR!!!:",x[index_min], y[index_min], values[index_min])
    return x[index_min], y[index_min]

path = '/Users/weugene/basilisk/work/tube/res8_fast/'
path_ex = '/Users/weugene/basilisk/work/tube/'
case = '8'
x_name = 'Points_0'
y_name = 'Points_1'
file_name = ["data_i=","_t=",".csv"]
list_of_consideration = range(0,137,1)
# list_of_consideration = [1, 30, 60, 96]
diam = 0.514
K_neigh = 3
# x_name = 'Points:0'
# y_name = 'Points:1'

files_csv = []
files_csv += glob.glob(path + file_name[0]+"*"+file_name[1]+"*"+file_name[2])
print("FOUND:",files_csv)
files_csv=[ os.path.basename(x) for x in files_csv]

files_nums=[]
for x in files_csv:
    match_number = re.compile('-?\ *[0-9]+\.?[0-9]*(?:[Ee]\ *-?\ *[0-9]+)?')
    final_list = re.findall(match_number, x)
    files_nums.append((int(final_list[0]), float(final_list[1]), x))

files_nums=sorted(files_nums, key=lambda x: x[0])
print ("FILES_NUMS after soting:",files_nums)
files_csv_sorted_filtered = []
files_csv = []
for (i,x) in enumerate(files_nums):
    if x[0] in list_of_consideration:
        files_csv_sorted_filtered.append((x[0], x[1] , path + x[2]))
        files_csv.append(path + x[2])
        print("Files csv are reading...", x[2])

Nf = len(files_csv)
def name_generator(s):
#     return  [path_ex + '10' + s + '.csv', path_ex + '10' + s + '_Fluent.csv', path_ex + '10' + s + '_OpenFoam.csv' ], ['10' + s + ' experiment', '10' + s + ' Fluent','10' + s + ' OpenFoam', '10' + s + ' BP']
#     return  [path_ex + '10' + s + '.csv', path_ex + '10' + s + '_Fluent_neighbors.csv', path_ex + '10' + s + '_OpenFoam.csv' ], ['10' + s + ' experiment', '10' + s + ' Fluent','10' + s + ' OpenFoam', '10' + s + ' BP']
    return  [path_ex + '5b_' + s + '.csv'],[ '5b_' + s ]


files_csv_exp, names = name_generator(case)

col =   ['blue',           'red',        'hsv(120,100,100)',       'black' ]
# files_csv_exp = ['a.csv','b.csv','c.csv']
print('files_csv:',files_csv)
print('files_csv_exp:',files_csv_exp)
df = []
df_exp = []
# params = ['10a.csv':'']
fig = go.Figure()

for i, file in enumerate(files_csv_exp):
    print('reading .. ', file)
    df_exp.append(pd.read_csv(file, header=None))
    df_exp[i].columns = [x_name, y_name] 
    df_exp[i][x_name] /= diam
    df_exp[i][y_name] /= diam

i_exp = 0
for i, file in enumerate(files_csv):
    print("file:",file,"\n")
    df.append(pd.read_csv(file))  
    mdf = df[i]
    j = str(files_csv_sorted_filtered[i][0])
    t = str(files_csv_sorted_filtered[i][1])
#     print('i=', j, ' t=', t,' file=', file)
#     display(mdf.head())
#     N = mdf.shape[0]
#     dfdown = mdf[(mdf[y_name]<0)].sort_values(by=[x_name, y_name])
#     dfup = mdf[(mdf[y_name]>0)].sort_values(by=[x_name, y_name], ascending=False)
#     mdf = pd.concat([dfdown, dfup, dfdown.head(1)])
    x = np.array(mdf[x_name][mdf[y_name]>=0])
    y = np.array(mdf[y_name][mdf[y_name]>=0])
    L = x.size
    print("size of x:", L)
    if L<5:
        print('ERROR: no points < 5!')
    aa = zip(x,y)
    
    icr = np.argmax(y)
    xcr = x[icr]
    ycr = y[icr]
    def get_item(item):
        x,y = item
        if x < xcr:
            return item
        else:
            return (item[0], -item[0])
        
    aa = sorted(aa, key=lambda item: get_item(item))
    x,y = zip(*aa)
    xx, yy = [], []
    for i,x0 in enumerate(x):
        y0 = y[i]
        a,b = nearest_point(x, y, x0, y0)
        xx.append(a)
        yy.append(b)
    x = xx[:]
    y = yy[:]
    y = non_uniform_savgol(x, y, 51, 2)
#     xx, yy = [], []
#     for j in range(0,L):
#         start = max(0,j-K_neigh)
#         end = min(L-1,j+K_neigh)
#         xx.append(mean(x[start:end]))
#         yy.append(mean(y[start:end]))
#     x = xx[:]
#     y = yy[:]

#     spl = splrep(x, y)
#     x2 = np.linspace(min(x), max(x), 200)
#     y2 = splev(x2, spl)
#     x = x2[:]
#     y = y2[:]

#     y = scipy.signal.savgol_filter(y, 31, 1) # window size 51, polynomial order 3
    
#     ius = InterpolatedUnivariateSpline(x, y) # x must increase
#     y = ius(x)

    
    # use RBF method
#     print('XY:',x,y)
#     xi = np.linspace(min(x), max(x), 100)
#     rbf = Rbf(x, y, function = 'gaussian')
#     y = rbf(xi)

    minx = min(x)
    print("Thickness=", 0.5 - max(y))
    fig.add_trace(go.Scatter(x=x, y=y, name='t=' + t,
                             mode='lines',
                             textfont=dict(
                                family="sans serif",
                                size=18,
                                color="LightSeaGreen")
                             ))
    if True:
        fig1 = go.Figure(go.Scatter(x=x-minx, y=y, name='t=' + t,
#                                  mode='markers',
#                                  mode='lines+markers',
                                 mode='lines',                                    
                                 marker = dict(
                                     color = col[-1],
                                     size = 5
        #                                  ,
        #                              line = dict(
        #                               color = 'rgb(231, 99, 250)',
        #                               width = 12
        #                               )
                                 ),
                                 textfont=dict(
                                    family="sans serif",
                                    size=18,
                                    color="LightSeaGreen")
                                 ))
        
        for i_exp in range(0,1):
            fig1.add_trace(go.Scatter(x=df_exp[i_exp][x_name], y=df_exp[i_exp][y_name], name=names[i_exp],
                                 mode='lines',
                                 marker = dict(
                                     color = col[i_exp],
                                     size = 5
    #                                  ,
        #                              line = dict(
        #                               color = 'rgb(231, 99, 250)',
        #                               width = 12
        #                               )
                                 ),
                                 textfont=dict(
                                    family="sans serif",
                                    size=18,
                                    color="LightSeaGreen")
                                 ))
        fig1.update_layout(
            width = 1000,
            height = 500,
            xaxis_title='x',
            yaxis_title='y'
        )

        fig1.update_yaxes(range=[0,0.5])
        fig1.update_layout(showlegend=False)
        iplot(fig1)
        pio.write_image(fig1, path + 'bubble_evolution_t=' + t + '.pdf')
    
    fig.add_trace(go.Scatter(
        x=[0, 100],
        y=[0.5, 0.5],
        mode='lines',
        line=dict(color='black', width=1),
        showlegend=False)
    )
    
#     fig.update_xaxes(range=[1.9,17.6])
    
    fig.add_trace(go.Scatter(
        x=[0, 100],
        y=[-0.5, -0.5],
        mode='lines',
        line=dict(color='black', width=1),
        showlegend=False)
    )
#     fig.add_trace(go.Scatter(x=[min(x), max(x)], y=[0.5,0.5], name='t='+str(i),
#                              mode='lines',
#                              textfont=dict(
#                                 family="sans serif",
#                                 size=18,
#                                 color="LightSeaGreen")

fig.update_layout(
    width = 1000,
    height = 300,
    xaxis_title='x',
    yaxis_title='y'
)
    
fig.update_yaxes(range=[0,0.6])
iplot(fig)
pio.write_image(fig, path + 'bubble_evolution.pdf')
# fig.show()

FOUND: ['/Users/weugene/basilisk/work/tube/res8_fast/data_i=3_t=0.602807.csv', '/Users/weugene/basilisk/work/tube/res8_fast/data_i=1_t=1.0.csv', '/Users/weugene/basilisk/work/tube/res8_fast/data_i=1_t=0.138782.csv', '/Users/weugene/basilisk/work/tube/res8_fast/data_i=0_t=0.0.csv', '/Users/weugene/basilisk/work/tube/res8_fast/data_i=2_t=0.370794.csv', '/Users/weugene/basilisk/work/tube/res8_fast/data_i=5_t=1.13597.csv', '/Users/weugene/basilisk/work/tube/res8_fast/data_i=10_t=1.6232.csv', '/Users/weugene/basilisk/work/tube/res8_fast/data_i=4_t=0.83482.csv', '/Users/weugene/basilisk/work/tube/res8_fast/data_i=6_t=1.25198.csv', '/Users/weugene/basilisk/work/tube/res8_fast/data_i=0_t=1.02507.csv', '/Users/weugene/basilisk/work/tube/res8_fast/data_i=3_t=3.0.csv', '/Users/weugene/basilisk/work/tube/res8_fast/data_i=0_t=0.00421461.csv', '/Users/weugene/basilisk/work/tube/res8_fast/data_i=9_t=1.6.csv', '/Users/weugene/basilisk/work/tube/res8_fast/data_i=7_t=1.36798.csv', '/Users/weugene/basili

file: /Users/weugene/basilisk/work/tube/res8_fast/data_i=0_t=1.02507.csv 

size of x: 276
Thickness= 0.106803514108949


file: /Users/weugene/basilisk/work/tube/res8_fast/data_i=0_t=0.00421461.csv 

size of x: 254
Thickness= 0.10600737899562762


file: /Users/weugene/basilisk/work/tube/res8_fast/data_i=1_t=1.0.csv 

size of x: 278
Thickness= 0.10490568151360158


file: /Users/weugene/basilisk/work/tube/res8_fast/data_i=1_t=0.138782.csv 

size of x: 266
Thickness= 0.10731018827732913


...

file: /Users/weugene/basilisk/work/tube/res8_fast/data_i=6_t=1.25198.csv 

size of x: 267
Thickness= 0.10698401556552661


file: /Users/weugene/basilisk/work/tube/res8_fast/data_i=7_t=1.36798.csv 

size of x: 263
Thickness= 0.10777547447407398


file: /Users/weugene/basilisk/work/tube/res8_fast/data_i=8_t=1.48399.csv 

size of x: 274
Thickness= 0.10690217425924903


file: /Users/weugene/basilisk/work/tube/res8_fast/data_i=9_t=1.6.csv 

size of x: 270
Thickness= 0.1039700911869158


file: /Users/weugene/basilisk/work/tube/res8_fast/data_i=10_t=1.6232.csv 

size of x: 262
Thickness= 0.10605695976362517
